In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os

# Load the MediaPipe Hands model
mp_hands = mp.solutions.hands.Hands()

def get_landmarks(file_location):
    # Read the image
    image = cv2.imread(file_location)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and get the landmarks data
    results = mp_hands.process(image_rgb)

    # Extract the landmarks from the results
    landmarks = results.multi_hand_landmarks
    
    # Convert the landmarks to a numpy array
    if landmarks:
        landmarks_np = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks[0].landmark])
    else:
        return None
    
    return landmarks_np

I0000 00:00:1719941063.398206 4480198 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro


In [2]:
# count the number of subfolders and files in the given 
files = []
folders = []
def count_files(path='.'):
    global file_count, folder_count
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                files.append(entry.path)
            elif entry.is_dir():
                folders.append(entry.path)
                count_files(entry.path)
        
    return files, folders

count_files("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

len(set(files)), len(set(folders))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719941063.414890 4480522 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719941063.421870 4480522 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


(223075, 29)

In [ ]:
# Initialize the list to store the landmarks
landmarks_list = []
        
# Initialize the list to store the labels
labels = []

global curr_label
curr_label = ""

def getLandmarksAndLabels(path='.'):
    global curr_label
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                landmarks = get_landmarks(entry.path)
                # print(landmarks)
                if landmarks is not None:
                    landmarks_list.append(landmarks)
                    labels.append(curr_label)
                # else:
                #     print("No landmarks found")
            elif entry.is_dir():
                curr_label = entry.name
                print(curr_label)
                getLandmarksAndLabels(entry.path)
        
    return landmarks_list, labels

x_train, y_train = getLandmarksAndLabels("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1719360603.128705 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719360603.134692 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/gvssriram/anaconda3/envs/math_note_x/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


R
U
I
N
G
Z
T
S
A
F
O
H
del
nothing
space
M
J
C
D
V
Q
X
E
B
K
L
Y
P
W


In [ ]:
type(x_train), type(y_train), type(x_train[0]), type(y_train[0])

(list, list, numpy.ndarray, str)

In [ ]:
# convert the list to numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

((169090, 21, 3), (169090,))

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train.npy', x_train)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/y_train.npy', y_train)

In [ ]:
# reshape x_train
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_train.shape

(169090, 63)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train_flat.npy', x_train)

## Load processed numpy image data

In [6]:
# Load numpy arrays
x_train = np.load('../data/ASL_Alphabet_Dataset_Processed/x_train.npy')
y_train = np.load('../data/ASL_Alphabet_Dataset_Processed/y_train.npy')

In [7]:
x_train.shape, y_train.shape

((169090, 21, 3), (169090,))

In [8]:
x_train

array([[[ 5.15921772e-01,  7.92883813e-01,  4.68830564e-07],
        [ 5.61473846e-01,  7.50161648e-01, -2.74555814e-02],
        [ 5.77734768e-01,  6.88524961e-01, -4.12738435e-02],
        ...,
        [ 4.53955889e-01,  6.46603763e-01, -9.78934020e-02],
        [ 4.90068585e-01,  6.74939871e-01, -1.07148461e-01],
        [ 5.13166964e-01,  7.01320827e-01, -1.07620858e-01]],

       [[ 5.05461454e-01,  8.36323082e-01,  1.18730497e-06],
        [ 4.40506786e-01,  7.34651744e-01, -7.65210986e-02],
        [ 4.25234973e-01,  5.97333670e-01, -9.95539799e-02],
        ...,
        [ 6.71594977e-01,  4.91894424e-01, -9.87732336e-02],
        [ 6.45163357e-01,  5.66112816e-01, -1.00976720e-01],
        [ 6.21084809e-01,  6.32574558e-01, -7.91223496e-02]],

       [[ 7.31024981e-01,  8.74174833e-01,  2.61961304e-07],
        [ 6.56271577e-01,  7.75128603e-01, -1.16133709e-02],
        [ 6.35274231e-01,  6.68298721e-01, -1.62141919e-02],
        ...,
        [ 7.70584047e-01,  6.44464433e-01,

In [10]:
# Each row has 21 points of landmarks (x,y,z). Compute the relative positions between each point and 0th point
# Compute the relative positions between each point and 0th point
def get_relative_positions(x):
    relative_positions = []
    for i in range(1, len(x)):
        relative_positions.append(x[i] - x[0])
    return np.array(relative_positions)

# Compute the relative positions between each point and 0th point
x_train_relative = np.apply_along_axis(get_relative_positions, 1, x_train)
x_train_relative.shape

np.save('../data/ASL_Alphabet_Dataset_Processed/x_train_relative.npy', x_train_relative)

# Load numpy arrays
x_train_relative = np.load('../data/ASL_Alphabet_Dataset_Processed/x_train_relative.npy')

x_train_relative.shape

x_train_relative

array([[[ 0.04555207, -0.04272217, -0.02745605],
        [ 0.061813  , -0.10435885, -0.04127431],
        [ 0.02810705, -0.13061208, -0.05686834],
        ...,
        [-0.06196588, -0.14628005, -0.09789387],
        [-0.02585319, -0.11794394, -0.10714893],
        [-0.00275481, -0.09156299, -0.10762133]],

       [[-0.06495467, -0.10167134, -0.07652229],
        [-0.08022648, -0.23898941, -0.09955517],
        [-0.01976028, -0.35343987, -0.12149395],
        ...,
        [ 0.16613352, -0.34442866, -0.09877442],
        [ 0.1397019 , -0.27021027, -0.10097791],
        [ 0.11562335, -0.20374852, -0.07912354]],

       [[-0.0747534 , -0.09904623, -0.01161363],
        [-0.09575075, -0.20587611, -0.01621445],
        [-0.06241727, -0.27738518, -0.03013196],
        ...,
        [ 0.03955907, -0.2297104 , -0.02239184],
        [ 0.00394905, -0.184394  , -0.01331543],
        [-0.00386137, -0.15110785,  0.00302288]],

       ...,

       [[ 0.07857627, -0.0745883 , -0.1162138 ],
        [ 0

In [11]:
x_train_relative.shape

(169090, 20, 3)

In [12]:
# one hot encode the labels
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1,1)).toarray()

In [13]:
y_train.shape

(169090, 29)

In [14]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
x_train_relative = x_train_relative.reshape(x_train_relative.shape[0], x_train_relative.shape[1]*x_train_relative.shape[2])
x_train_relative.shape

(169090, 60)

In [16]:
# Create a classification model using tensorflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Dense(128, input_shape=(x_train_relative.shape[1],), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(29, activation='softmax'))

model.summary()

opt = tf.keras.optimizers.Adam(learning_rate=0.005)

loss = tf.keras.losses.CategoricalCrossentropy()

metrics = []
metrics.append("categorical_accuracy")

# callbacks

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint(filepath='../models/ckpts/asl_alphabet_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001)
tensorboard = TensorBoard(log_dir='../tflogs')

callbacks = [early_stopping, model_checkpoint, reduce_lr, tensorboard]

model.compile(optimizer=opt, loss=loss, metrics=metrics)

history = model.fit(x_train_relative, y_train, epochs=100, validation_split=0.2, verbose=1, callbacks=callbacks)

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         7,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │           957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,997 (78.11 KB)

 Trainable params: 19,549 (76.36 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 5s 836us/step - categorical_accuracy: 0.4676 - loss: 2.3455 - val_categorical_accuracy: 0.0286 - val_loss: 10.9783 - learning_rate: 0.0050
Epoch 2/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 812us/step - categorical_accuracy: 0.5840 - loss: 1.7333 - val_categorical_accuracy: 0.0259 - val_loss: 12.0989 - learning_rate: 0.0050
Epoch 3/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 789us/step - categorical_accuracy: 0.5860 - loss: 1.7120 - val_categorical_accuracy: 0.0275 - val_loss: 14.8708 - learning_rate: 0.0050
Epoch 4/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 778us/step - categorical_accuracy: 0.5810 - loss: 1.6994 - val_categorical_accuracy: 0.0298 - val_loss: 17.4214 - learning_rate: 0.0050
Epoch 5/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 772us/step - categorical_accuracy: 0.6659 - loss: 1.3594 - val_categorical_accuracy: 0.0296 - val_loss: 18.7067 - learning_rate: 5.0000e-04
Epoch 6/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 784us/step - categorical_accuracy: 0.703

In [18]:
y_pred = model.predict(x_train_relative)

5285/5285 ━━━━━━━━━━━━━━━━━━━━ 2s 283us/step


In [19]:
y_pred[0]

array([2.7921494e-06, 6.5810517e-03, 1.4306526e-04, 1.4486490e-04,
       1.8309147e-04, 7.0842635e-04, 4.2900341e-05, 3.5214867e-03,
       1.5406873e-07, 3.2711628e-06, 1.6626106e-09, 1.7903333e-09,
       1.3582588e-05, 6.6933762e-06, 2.2234150e-05, 1.7186821e-09,
       2.2016544e-05, 1.9907267e-01, 1.0864065e-04, 1.1295699e-04,
       5.4506183e-01, 2.4385253e-01, 1.8903989e-09, 3.1724933e-04,
       1.8134144e-09, 5.1291594e-05, 5.8289618e-07, 3.4878096e-07,
       2.6169939e-05], dtype=float32)

In [20]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([20, 17, 17, ..., 20, 20, 20])

In [21]:
y_train_labels = np.argmax(y_train, axis=1)
y_train_labels

array([17, 17, 17, ..., 22, 22, 22])

In [22]:
# Evaluate the model

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy_score(y_train_labels, y_pred)

0.7038618487196168

In [23]:
print(classification_report(y_train_labels, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83      6087
           1       0.78      0.98      0.87      6931
           2       0.93      0.92      0.92      5740
           3       0.51      0.85      0.63      6890
           4       0.94      0.92      0.93      6240
           5       0.95      0.96      0.96      7514
           6       0.83      0.93      0.88      6334
           7       0.74      0.92      0.82      6407
           8       0.67      0.90      0.77      6700
           9       0.65      0.89      0.75      5767
          10       0.00      0.00      0.00      7097
          11       0.00      0.00      0.00      7294
          12       0.65      0.72      0.68      4123
          13       0.70      0.51      0.59      3579
          14       0.81      0.94      0.87      6254
          15       0.00      0.00      0.00      5224
          16       0.56      0.96      0.71      5223
          17       0.61    

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [ ]:
# Release the MediaPipe Hands model
mp_hands.close()